# HW: X-ray images classification
--------------------------------------

Before you begin, open Mobaxterm and connect to triton with the user and password you were give with. Activate the environment `2ndPaper` and then type the command `pip install scikit-image`.

In this assignment you will be dealing with classification of 32X32 X-ray images of the chest. The image can be classified into one of four options: lungs (l), clavicles (c), and heart (h) and background (b). Even though those labels are dependent, we will treat this task as multiclass and not as multilabel. The dataset for this assignment is located on a shared folder on triton (`/MLdata/MLcourse/X_ray/'`).

In [1]:
import os
import numpy as np
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Dropout
from tensorflow.keras.layers import Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import *

from tensorflow.keras.initializers import Constant
from tensorflow.keras.datasets import fashion_mnist
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from skimage.io import imread

from skimage.transform import rescale, resize, downscale_local_mean
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options =
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
def preprocess(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        src = imread(os.path.join(datapath, fn),1)
        img = resize(src,(32,32),order = 3)
        
        images[ii,:,:,0] = img
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    BaseImages = images
    BaseY = Y
    return BaseImages, BaseY

In [4]:
def preprocess_train_and_val(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        images[ii,:,:,0] = imread(os.path.join(datapath, fn),1)
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    return images, Y

In [53]:
#Loading the data for training and validation:
src_data = 'X_ray/' #/MLdata/MLcourse/
train_path = src_data + 'train'
val_path = src_data + 'validation'
test_path = src_data + 'test'
BaseX_train , BaseY_train = preprocess_train_and_val(train_path)
BaseX_val , BaseY_val = preprocess_train_and_val(val_path)
X_test, Y_test = preprocess(test_path)

In [6]:
keras.backend.clear_session()

### PART 1: Fully connected layers 
--------------------------------------

---
<span style="color:red">***Task 1:***</span> *NN with fully connected layers. 

Elaborate a NN with 2 hidden fully connected layers with 300, 150 neurons and 4 neurons for classification. Use ReLU activation functions for the hidden layers and He_normal for initialization. Don't forget to flatten your image before feedforward to the first dense layer. Name the model `model_relu`.*

---

In [71]:
#--------------------------Impelment your code here:-------------------------------------
n_hidden_start=300
len_sub_window=10
input_shape = (32,32,1)
model_relu = Sequential()
model_relu.add(Flatten(input_shape=(32,32,)))
model_relu.add(Dense(n_hidden_start,kernel_initializer="HeNormal"))
model_relu.add(Dense(int(n_hidden_start/2), activation='relu'))
model_relu.add(Dense(int(n_hidden_start/75), activation='relu'))

#----------------------------------------------------------------------------------------

In [72]:
model_relu.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_14 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dense_37 (Dense)             (None, 300)               307500    
_________________________________________________________________
dense_38 (Dense)             (None, 150)               45150     
_________________________________________________________________
dense_39 (Dense)             (None, 4)                 604       
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
_________________________________________________________________


In [73]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)


Compile the model with the optimizer above, accuracy metric and adequate loss for multiclass task. Train your model on the training set and evaluate the model on the testing set. Print the accuracy and loss over the testing set.

In [86]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='binary_crossentropy')
history_relu = model_relu.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs)

y_pred_test = model_relu.predict(X_test)
print("Evaluate on test data")
results = model_relu.evaluate(X_test, Y_test, batch_size=batch_size)
print("test loss, test acc:", results)

#----------------------------------------------------------------------------------------

Epoch 1/25
102/102 [==============================] - 1s 2ms/step - loss: 0.2042 - accuracy: 0.8794
Epoch 2/25
102/102 [==============================] - 0s 3ms/step - loss: 0.2060 - accuracy: 0.8774
Epoch 3/25
102/102 [==============================] - 0s 2ms/step - loss: 0.1982 - accuracy: 0.8785
Epoch 4/25
102/102 [==============================] - 0s 2ms/step - loss: 0.1908 - accuracy: 0.8813
Epoch 5/25
102/102 [==============================] - 0s 2ms/step - loss: 0.2444 - accuracy: 0.8611
Epoch 6/25
102/102 [==============================] - 0s 2ms/step - loss: 0.1971 - accuracy: 0.8816
Epoch 7/25
102/102 [==============================] - 0s 2ms/step - loss: 0.2070 - accuracy: 0.8679
Epoch 8/25
102/102 [==============================] - 0s 2ms/step - loss: 0.1979 - accuracy: 0.8731
Epoch 9/25
102/102 [==============================] - 0s 2ms/step - loss: 0.1965 - accuracy: 0.8829
Epoch 10/25
102/102 [==============================] - 0s 2ms/step - loss: 0.2100 - accuracy: 0.8752

---
<span style="color:red">***Task 2:***</span> *Activation functions.* 

Change the activation functions to LeakyRelu or tanh or sigmoid. Name the new model `new_a_model`. Explain how it can affect the model.*

---

In [136]:
#--------------------------Impelment your code here:-------------------------------------
n_hidden_start=300
len_sub_window=10
input_shape = (32,32,1)
new_a_model = Sequential()
new_a_model.add(Flatten(input_shape=(32,32,)))
new_a_model.add(Dense(n_hidden_start,kernel_initializer="HeNormal"))
new_a_model.add(Dense(int(n_hidden_start/2), activation='sigmoid'))
new_a_model.add(Dense(int(n_hidden_start/75), activation='sigmoid'))

#----------------------------------------------------------------------------------------

In [137]:
new_a_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 300)               307500    
_________________________________________________________________
dense_15 (Dense)             (None, 150)               45150     
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 604       
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
_________________________________________________________________


---
<span style="color:red">***Task 3:***</span> *Number of epochs.* 

Train the new model using 25 and 40 epochs. What difference does it makes in term of performance? Remember to save the compiled model for having initialized weights for every run as we did in tutorial 12. Evaluate each trained model on the test set*

---

In [138]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)



In [139]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='binary_crossentropy')

if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "init_weigths_newmodel"
model_path = os.path.join(save_dir, model_name)
new_a_model.save(model_path)
# print('Saved initialized model at %s ' % model_path)

history_model_a = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs)

y_pred_test = new_a_model.predict(X_test)
print("Evaluate on test data")
results = new_a_model.evaluate(X_test, Y_test, batch_size=batch_size)
print("test loss, test acc:", results)

#-----------------------------------------------------------------------------------------

Epoch 1/25
102/102 [==============================] - 1s 2ms/step - loss: 0.6970 - accuracy: 0.2486
Epoch 2/25
102/102 [==============================] - 0s 2ms/step - loss: 0.5601 - accuracy: 0.4419
Epoch 3/25
102/102 [==============================] - 0s 2ms/step - loss: 0.5342 - accuracy: 0.5271
Epoch 4/25
102/102 [==============================] - 0s 2ms/step - loss: 0.5214 - accuracy: 0.5374
Epoch 5/25
102/102 [==============================] - 0s 2ms/step - loss: 0.5086 - accuracy: 0.5705
Epoch 6/25
102/102 [==============================] - 0s 2ms/step - loss: 0.4965 - accuracy: 0.5774
Epoch 7/25
102/102 [==============================] - 0s 2ms/step - loss: 0.4875 - accuracy: 0.5891
Epoch 8/25
102/102 [==============================] - 0s 2ms/step - loss: 0.4784 - accuracy: 0.5975
Epoch 9/25
102/102 [==============================] - 0s 2ms/step - loss: 0.4692 - accuracy: 0.6101
Epoch 10/25
102/102 [==============================] - 0s 2ms/step - loss: 0.4589 - accuracy: 0.6248

In [96]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 40

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)



In [97]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='binary_crossentropy')

new_a_model = load_model("results/init_weigths_newmodel") # Initializing weights before total run

history_model_a = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs)

y_pred_test = new_a_model.predict(X_test)
print("Evaluate on test data")
results = new_a_model.evaluate(X_test, Y_test, batch_size=batch_size)
print("test loss, test acc:", results)
#-----------------------------------------------------------------------------------------

Epoch 1/40
102/102 [==============================] - 1s 2ms/step - loss: 0.2999 - accuracy: 0.8031
Epoch 2/40
102/102 [==============================] - 0s 2ms/step - loss: 0.3177 - accuracy: 0.7804
Epoch 3/40
102/102 [==============================] - 0s 2ms/step - loss: 0.3091 - accuracy: 0.7909
Epoch 4/40
102/102 [==============================] - 0s 2ms/step - loss: 0.2904 - accuracy: 0.8072
Epoch 5/40
102/102 [==============================] - 0s 2ms/step - loss: 0.2880 - accuracy: 0.8148
Epoch 6/40
102/102 [==============================] - 0s 2ms/step - loss: 0.2952 - accuracy: 0.8125
Epoch 7/40
102/102 [==============================] - 0s 2ms/step - loss: 0.2901 - accuracy: 0.8209
Epoch 8/40
102/102 [==============================] - 0s 2ms/step - loss: 0.2938 - accuracy: 0.8072
Epoch 9/40
102/102 [==============================] - 0s 2ms/step - loss: 0.2892 - accuracy: 0.8119
Epoch 10/40
102/102 [==============================] - 0s 2ms/step - loss: 0.3072 - accuracy: 0.7821

---
<span style="color:red">***Task 4:***</span> *Mini-batches.* 

Build the `model_relu` again and run it with a batch size of 32 instead of 64. What are the advantages of the mini-batch vs. SGD?*

---

In [ ]:
keras.backend.clear_session()

In [98]:
#--------------------------Impelment your code here:-------------------------------------
n_hidden_start=300
len_sub_window=10
input_shape = (32,32,1)
model_relu = Sequential()
model_relu.add(Flatten(input_shape=(32,32,)))
model_relu.add(Dense(n_hidden_start,kernel_initializer="HeNormal"))
model_relu.add(Dense(int(n_hidden_start/2), activation='relu'))
model_relu.add(Dense(int(n_hidden_start/75), activation='relu'))
#----------------------------------------------------------------------------------------

In [99]:
batch_size = 32
epochs = 50

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)


In [100]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='binary_crossentropy')
history_relu = model_relu.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs)

y_pred_test = model_relu.predict(X_test)
print("Evaluate on test data")
results = model_relu.evaluate(X_test, Y_test, batch_size=batch_size)
print("test loss, test acc:", results)
#----------------------------------------------------------------------------------------

Epoch 1/50
203/203 [==============================] - 1s 2ms/step - loss: 2.3412 - accuracy: 0.3358
Epoch 2/50
203/203 [==============================] - 0s 2ms/step - loss: 1.1903 - accuracy: 0.4925
Epoch 3/50
203/203 [==============================] - 0s 2ms/step - loss: 1.1043 - accuracy: 0.5390
Epoch 4/50
203/203 [==============================] - 0s 2ms/step - loss: 1.0044 - accuracy: 0.5649
Epoch 5/50
203/203 [==============================] - 0s 2ms/step - loss: 0.5005 - accuracy: 0.5852
Epoch 6/50
203/203 [==============================] - 0s 2ms/step - loss: 0.4749 - accuracy: 0.6214
Epoch 7/50
203/203 [==============================] - 1s 3ms/step - loss: 0.4409 - accuracy: 0.6656
Epoch 8/50
203/203 [==============================] - 0s 2ms/step - loss: 0.4514 - accuracy: 0.6680
Epoch 9/50
203/203 [==============================] - 0s 2ms/step - loss: 0.4200 - accuracy: 0.6839
Epoch 10/50
203/203 [==============================] - 0s 2ms/step - loss: 0.4196 - accuracy: 0.6833

---
<span style="color:red">***Task 4:***</span> *Batch normalization.* 

Build the `new_a_model` again and add batch normalization layers. How does it impact your results?*

---

In [101]:
keras.backend.clear_session()

In [140]:
#--------------------------Impelment your code here:-------------------------------------
n_hidden_start=300
len_sub_window=10
input_shape = (32,32,1)
new_a_model = Sequential()
new_a_model.add(Flatten(input_shape=(32,32,)))
new_a_model.add(Dense(n_hidden_start,kernel_initializer="HeNormal"))
new_a_model.add(BatchNormalization())
new_a_model.add(Dense(int(n_hidden_start/2), activation='sigmoid'))
new_a_model.add(BatchNormalization())
new_a_model.add(Dense(int(n_hidden_start/75), activation='sigmoid'))
new_a_model.add(BatchNormalization())
#---------------------------------------------------------------------------------------

In [141]:
batch_size = 32
epochs = 50

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)
#Compile the network: 


In [142]:
#Preforming the training by using fit 
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='binary_crossentropy')
history_model_a = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs)

y_pred_test = new_a_model.predict(X_test)
print("Evaluate on test data")
results = new_a_model.evaluate(X_test, Y_test, batch_size=batch_size)
print("test loss, test acc:", results)
#----------------------------------------------------------------------------------------

Epoch 1/50
203/203 [==============================] - 1s 2ms/step - loss: 3.8640 - accuracy: 0.3175
Epoch 2/50
203/203 [==============================] - 0s 2ms/step - loss: 3.0330 - accuracy: 0.4735
Epoch 3/50
203/203 [==============================] - 0s 2ms/step - loss: 2.8181 - accuracy: 0.5012
Epoch 4/50
203/203 [==============================] - 0s 2ms/step - loss: 2.6548 - accuracy: 0.5353
Epoch 5/50
203/203 [==============================] - 0s 2ms/step - loss: 2.4340 - accuracy: 0.5758
Epoch 6/50
203/203 [==============================] - 0s 2ms/step - loss: 2.3151 - accuracy: 0.5795
Epoch 7/50
203/203 [==============================] - 0s 2ms/step - loss: 2.1752 - accuracy: 0.5999
Epoch 8/50
203/203 [==============================] - 0s 2ms/step - loss: 2.0548 - accuracy: 0.6088
Epoch 9/50
203/203 [==============================] - 0s 2ms/step - loss: 2.0819 - accuracy: 0.6053
Epoch 10/50
203/203 [==============================] - 0s 2ms/step - loss: 2.1086 - accuracy: 0.6080

### PART 2: Convolutional Neural Network (CNN)
------------------------------------------------------------------------------------

---
<span style="color:red">***Task 1:***</span> *2D CNN.* 

Have a look at the model below and answer the following:
* How many layers does it have?
* How many filter in each layer?
* Would the number of parmaters be similar to a fully connected NN?
* Is this specific NN performing regularization?

---

In [111]:
def get_net(input_shape,drop,dropRate,reg):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model

In [112]:
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 1e-03
batch_size = 64
epochs = 25
drop = True
dropRate = 0.3
reg = 1e-2
NNet = get_net(input_shape,drop,dropRate,reg)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 128)      

In [113]:
NNet=get_net(input_shape,drop,dropRate,reg)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_7 (Dropout)          (None, 16, 16, 128)      

In [114]:
from tensorflow.keras.optimizers import *
import os
from tensorflow.keras.callbacks import *

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

#Compile the network: 
NNet.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')

#Saving checkpoints during training:
Checkpath = os.getcwd()
Checkp = ModelCheckpoint(Checkpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, save_freq=1)

In [115]:
#Preforming the training by using fit 
# IMPORTANT NOTE: This will take a few minutes!
h = NNet.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
#NNet.save(model_fn)

Epoch 1/25
102/102 [==============================] - 50s 482ms/step - loss: 8.3069 - acc: 0.3611 - val_loss: 7.9334 - val_acc: 0.2500
Epoch 2/25
102/102 [==============================] - 51s 500ms/step - loss: 7.5766 - acc: 0.5387 - val_loss: 8.0772 - val_acc: 0.2500
Epoch 3/25
102/102 [==============================] - 50s 494ms/step - loss: 7.4409 - acc: 0.5672 - val_loss: 8.2559 - val_acc: 0.2500
Epoch 4/25
102/102 [==============================] - 50s 493ms/step - loss: 7.3691 - acc: 0.5963 - val_loss: 8.2831 - val_acc: 0.2569
Epoch 5/25
102/102 [==============================] - 51s 502ms/step - loss: 7.2492 - acc: 0.6337 - val_loss: 8.1505 - val_acc: 0.3084
Epoch 6/25
102/102 [==============================] - 50s 493ms/step - loss: 7.1655 - acc: 0.6514 - val_loss: 7.9588 - val_acc: 0.3229
Epoch 7/25
102/102 [==============================] - 51s 500ms/step - loss: 7.0958 - acc: 0.6800 - val_loss: 7.8131 - val_acc: 0.3530
Epoch 8/25
102/102 [==============================] - 5

In [116]:
# NNet.load_weights('Weights_1.h5')

In [117]:
results = NNet.evaluate(X_test,Y_test)
print('test loss, test acc:', results)

6/6 [==============================] - 0s 42ms/step - loss: 8.0024 - acc: 0.3086
test loss, test acc: [8.00239086151123, 0.3085714280605316]


---
<span style="color:red">***Task 2:***</span> *Number of filters* 

Rebuild the function `get_net` to have as an input argument a list of number of filters in each layers, i.e. for the CNN defined above the input should have been `[64, 128, 128, 256, 256]`. Now train the model with the number of filters reduced by half. What were the results.

---

In [120]:
#--------------------------Impelment your code here:-------------------------------------
def get_net1(input_shape,drop,dropRate,reg,num_filters):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=num_filters[0], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=num_filters[1], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=num_filters[2], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=num_filters[3], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=num_filters[4], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model

num_filters = [32, 64, 64, 128, 128]
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 1e-03
batch_size = 64
epochs = 25
drop = True
dropRate = 0.3
reg = 1e-2
NNet_new = get_net1(input_shape,drop,dropRate,reg,num_filters)

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

#Compile the network: 
NNet_new.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')

#Saving checkpoints during training:
Checkpath = os.getcwd()
Checkp = ModelCheckpoint(Checkpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, save_freq=1)
h = NNet_new.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
results = NNet_new.evaluate(X_test,Y_test)
print('test loss, test acc:', results)
#----------------------------------------------------------------------------------------

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_4 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 256)       2560      
_________________________________________________________________
dropout_24 (Dropout)         (None, 32, 32, 256)       0         
_________________________________________________________________
batch_normalization_29 (Batc (None, 32, 32, 256)       128       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 16, 16, 256)       0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
dropout_25 (Dropout)         (None, 16, 16, 256)      

That's all folks! See you :)